In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))


In [ ]:
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d mohammedalbrayh/images-for-saudi-arabia-landmarks --unzip -p /content/saudi_landmarks


In [ ]:
!pip install faiss-cpu --quiet


In [ ]:
from pathlib import Path
from PIL import Image
import torch
import numpy as np
import faiss
from transformers import CLIPProcessor, CLIPModel

images_dir = Path("/content/saudi_landmarks")
img_paths = list(images_dir.rglob("*.png")) + list(images_dir.rglob("*.jpg"))
print("Found", len(img_paths), "images")


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
embs = []
meta = []
for p in img_paths:
    img = Image.open(p).convert("RGB")
    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        e = model.get_image_features(**inputs).cpu().numpy().squeeze()
    embs.append(e)
    meta.append(p)
embs = np.stack(embs)

In [ ]:
index = faiss.IndexFlatL2(embs.shape[1])
index.add(embs)
print("✅ FAISS index built")

In [ ]:
query = "Diriyah"
q_inputs = processor(text=query, return_tensors="pt").to(device)
with torch.no_grad():
    q_emb = model.get_text_features(**q_inputs).cpu().numpy()
D, I = index.search(q_emb, k=5)

In [ ]:
for rank, idx in enumerate(I[0]):
    print(f"{rank+1}. {meta[idx].parent.name} / {meta[idx].name} (distance={D[0][rank]:.2f})")
    display(Image.open(meta[idx]).resize((256,256)))

In [ ]:
query = "The Clock Towers"
q_inputs = processor(text=query, return_tensors="pt").to(device)

with torch.no_grad():
    q_emb = model.get_text_features(**q_inputs).cpu().numpy()

D, I = index.search(q_emb, k=3)

print(f"Top-5 landmarks similar to '{query}':")
for rank, idx in enumerate(I[0]):
    print(f"{rank+1}. {meta[idx].parent.name} / {meta[idx].name} (distance={D[0][rank]:.2f})")
    display(Image.open(meta[idx]).resize((256, 256)))

In [ ]:
import json
from pathlib import Path

landmark_folders = [
    "The Kaaba",
    "Jabal AlFil (Elephant Rock)",
    "Kingdom Tower",
    "Ibrahim Palace",
    "Al Masmak Palace",
    "Al Rahmah Mosque",
    "Diriyah",
    "King Abdullah Financial District (KAFD)",
    "Maraya",
    "Nassif House Museum",
    "Riyadh Water Tower"
]

descriptions = {
    "The Kaaba": (
        "The Kaaba is a cuboid structure located inside the Masjid al-Haram in Mecca. "
        "It is the most sacred site in Islam and the direction (qibla) Muslims face during prayer.",
        [
            ("What shape is the Kaaba?", "cuboid", 16),
            ("Where is the Kaaba located?", "Masjid al-Haram in Mecca", 43),
            ("What is the Kaaba's significance in Islam?", "the most sacred site in Islam", 71)
        ]
    ),
    "Jabal AlFil (Elephant Rock)": (
        "Jabal AlFil, also known as Elephant Rock, is a natural rock formation in AlUla, Saudi Arabia. "
        "It resembles an elephant with a trunk touching the ground and is a popular tourist attraction.",
        [
            ("What is another name for Jabal AlFil?", "Elephant Rock", 25),
            ("Where is Jabal AlFil located?", "AlUla, Saudi Arabia", 72),
            ("Why is it called Elephant Rock?", "resembles an elephant with a trunk", 101)
        ]
    ),
    "Kingdom Tower": (
        "The Kingdom Tower is a skyscraper in Riyadh. It is one of the most iconic buildings in Saudi Arabia and features a skybridge with panoramic views.",
        [
            ("Where is Kingdom Tower located?", "Riyadh", 43),
            ("What does Kingdom Tower feature?", "a skybridge with panoramic views", 113)
        ]
    ),
    "Ibrahim Palace": (
        "Ibrahim Palace is a historical fort located in Al-Ahsa. It combines Islamic and military architectural styles and served as a government center during the Ottoman era.",
        [
            ("What type of architecture is seen in Ibrahim Palace?", "Islamic and military", 63),
            ("What was Ibrahim Palace used for?", "government center during the Ottoman era", 109)
        ]
    ),
    "Al Masmak Palace": (
        "Al Masmak Palace is a clay and mud-brick fort in Riyadh. It played a significant role in the unification of Saudi Arabia in the early 20th century.",
        [
            ("What material is Al Masmak Palace built from?", "clay and mud-brick", 36),
            ("Why is Al Masmak Palace historically important?", "significant role in the unification of Saudi Arabia", 74)
        ]
    ),
    "Al Rahmah Mosque": (
        "Al Rahmah Mosque is located in Jeddah, on the coast of the Red Sea. It is often called the Floating Mosque due to its structure over the water.",
        [
            ("Where is Al Rahmah Mosque located?", "Jeddah", 34),
            ("Why is Al Rahmah Mosque called the Floating Mosque?", "its structure over the water", 106)
        ]
    ),
    "Diriyah": (
        "Diriyah is a historic town in Saudi Arabia, recognized as a UNESCO World Heritage Site. It was the original home of the Saudi royal family and features traditional Najdi architecture.",
        [
            ("What is Diriyah recognized as?", "a UNESCO World Heritage Site", 52),
            ("What architectural style is found in Diriyah?", "Najdi architecture", 144)
        ]
    ),
    "King Abdullah Financial District (KAFD)": (
        "The King Abdullah Financial District is a business hub in Riyadh, known for its modern skyscrapers, eco-friendly buildings, and financial institutions.",
        [
            ("Where is the KAFD located?", "Riyadh", 52),
            ("What is KAFD known for?", "modern skyscrapers, eco-friendly buildings, and financial institutions", 67)
        ]
    ),
    "Maraya": (
        "Maraya is the world's largest mirrored building, located in the AlUla desert. It reflects the surrounding landscape and is used for concerts and cultural events.",
        [
            ("What makes Maraya unique?", "the world's largest mirrored building", 13),
            ("Where is Maraya located?", "AlUla desert", 58),
            ("What is Maraya used for?", "concerts and cultural events", 125)
        ]
    ),
    "Nassif House Museum": (
        "Nassif House Museum is a historical building in Jeddah’s Al-Balad district. It served as a royal guesthouse and features traditional Hijazi architecture.",
        [
            ("Where is Nassif House Museum located?", "Jeddah’s Al-Balad district", 47),
            ("What kind of architecture does Nassif House feature?", "Hijazi architecture", 121)
        ]
    ),
    "Riyadh Water Tower": (
        "The Riyadh Water Tower is a landmark structure in the city of Riyadh. It provides water storage and offers panoramic views from its upper levels.",
        [
            ("What is the function of Riyadh Water Tower?", "water storage", 72),
            ("What feature does the top of the tower offer?", "panoramic views", 103)
        ]
    )
}

squad_data = {"data": []}

for title, (context, qas_list) in descriptions.items():
    paragraph = {
        "context": context,
        "qas": [
            {
                "id": f"{title[:10]}_{i+1}",
                "question": q,
                "answers": [{"text": a, "answer_start": s}]
            }
            for i, (q, a, s) in enumerate(qas_list)
        ]
    }
    squad_data["data"].append({"title": title, "paragraphs": [paragraph]})
output_path = "/content/saudi_landmarks_qa.json"

with open(output_path, "w") as f:
    json.dump(squad_data, f, indent=2)

output_path


In [ ]:
import os

os.makedirs("/content", exist_ok=True)

output_path = "/content/saudi_landmarks_qa.json"
with open(output_path, "w") as f:
    json.dump(squad_data, f, indent=2)

output_path


In [ ]:
!pip install transformers datasets --quiet


In [ ]:
import json
from datasets import Dataset

with open("saudi_landmarks_qa.json", "r") as f:
    raw = json.load(f)

examples = []
for article in raw["data"]:
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            for ans in qa["answers"]:
                examples.append({
                    "id": qa["id"],
                    "question": qa["question"],
                    "context": context,
                    "answers": {"text": [ans["text"]], "answer_start": [ans["answer_start"]]}
                })

dataset = Dataset.from_list(examples)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")


In [ ]:
def preprocess(example):
    inputs = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=384,
        return_offsets_mapping=True
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_char = example["answers"]["answer_start"][0]
    end_char = start_char + len(example["answers"]["text"][0])

    start_token = end_token = 0
    for idx, (start, end) in enumerate(offset_mapping):
        if start <= start_char < end:
            start_token = idx
        if start < end_char <= end:
            end_token = idx

    inputs["start_positions"] = start_token
    inputs["end_positions"] = end_token
    return inputs


In [ ]:
tokenized_dataset = dataset.map(preprocess)


In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./saudi-qa-model",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    save_total_limit=2
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


In [ ]:
# Save model and tokenizer
model_path = "./saudi-landmark-qa"

trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)


In [ ]:
from transformers import pipeline

qa = pipeline("question-answering", model=model_path, tokenizer=model_path)

result = qa(
    question="What is Diriyah recognized as?",
    context="Diriyah is a historic town in Saudi Arabia, recognized as a UNESCO World Heritage Site."
)
print(result["answer"])


In [ ]:
from transformers import pipeline

qa = pipeline("question-answering", model="./saudi-landmark-qa", tokenizer="./saudi-landmark-qa")

# QA examples for each landmark
qa_examples = [
    {
        "context": "The Kaaba is a cuboid structure located inside the Masjid al-Haram in Mecca. It is the most sacred site in Islam and the direction (qibla) Muslims face during prayer.",
        "questions": [
            "What shape is the Kaaba?",
            "Where is the Kaaba located?"
        ]
    },
    {
        "context": "Jabal AlFil, also known as Elephant Rock, is a natural rock formation in AlUla, Saudi Arabia. It resembles an elephant with a trunk touching the ground and is a popular tourist attraction.",
        "questions": [
            "What does Jabal AlFil resemble?",
            "Where is Elephant Rock located?"
        ]
    },
    {
        "context": "The Kingdom Tower is a skyscraper in Riyadh. It is one of the most iconic buildings in Saudi Arabia and features a skybridge with panoramic views.",
        "questions": [
            "What city is the Kingdom Tower in?",
            "What does Kingdom Tower offer at the top?"
        ]
    },
    {
        "context": "Ibrahim Palace is a historical fort located in Al-Ahsa. It combines Islamic and military architectural styles and served as a government center during the Ottoman era.",
        "questions": [
            "Where is Ibrahim Palace located?",
            "What is unique about Ibrahim Palace’s architecture?"
        ]
    },
    {
        "context": "Al Masmak Palace is a clay and mud-brick fort in Riyadh. It played a significant role in the unification of Saudi Arabia in the early 20th century.",
        "questions": [
            "What material is Al Masmak Palace made of?",
            "What is the historical importance of Al Masmak Palace?"
        ]
    },
    {
        "context": "Al Rahmah Mosque is located in Jeddah, on the coast of the Red Sea. It is often called the Floating Mosque due to its structure over the water.",
        "questions": [
            "Why is Al Rahmah Mosque called the floating mosque?",
            "In which city is Al Rahmah Mosque located?"
        ]
    },
    {
        "context": "Diriyah is a historic town in Saudi Arabia, recognized as a UNESCO World Heritage Site. It was the original home of the Saudi royal family and features traditional Najdi architecture.",
        "questions": [
            "What is Diriyah recognized as?",
            "What architectural style is found in Diriyah?"
        ]
    }
]

qa_results = []
for entry in qa_examples:
    context = entry["context"]
    for q in entry["questions"]:
        answer = qa(question=q, context=context)["answer"]
        qa_results.append((q, answer))

import pandas as pd
df = pd.DataFrame(qa_results, columns=["Question", "Answer"])
print(df.to_markdown(index=False))


In [ ]:
!zip -r saudi-landmark-qa.zip ./saudi-landmark-qa
